In [1]:
import numpy as np
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.extmath import randomized_svd

#SVD

In [2]:
def svdd(X,k):
  U, Sigma, VT = randomized_svd(X, n_components=k)

# Reconstruct the original matrix using the truncated SVD components
  X_svd = U.dot(np.diag(Sigma)).dot(VT)

# Compare the reconstructed matrix with the original matrix
  return X_svd

#WOE+IV

In [3]:
def WOE_IV(lst, cls, toOcur)->list:
  #print(lst)
  #print(cls)
  total = len(lst)
  occuranceTotal = 0 
  counts = {}
  occurances = {}
  woe = {}
  iv = {}
  minValue = round(min(lst))
  maxValue = round(max(lst))
  dif = maxValue - minValue

  if dif>=100:
    noofBins =2
    if dif<500: noofBins=5
    else: noofBins=10 
    limits = dif//noofBins
    ranges = []
    for i in range(minValue+limits,maxValue,limits):
      ranges.append(i)
      counts.update({i:0.0})
      occurances.update({i:0.0})
      woe.update({i:0.0})
      iv.update({i:0.0})
    ranges.append(maxValue)
    counts.update({maxValue:0.0})
    occurances.update({maxValue:0.0})
    woe.update({maxValue:0.0})
    iv.update({maxValue:0.0})
    #print(ranges)

  else:
    ranges = list(set(lst))
    for i in ranges:
      counts.update({i:0.0})
      occurances.update({i:0.0})
      woe.update({i:0.0})
      iv.update({i:0.0})

  for element, c in zip(lst,cls):
    for rng in ranges:
      if element<=rng:
        counts[rng]+=1
        if c==toOcur:
          occurances[rng]+=1
          occuranceTotal+=1
        break    
    
  nonOccuranceTotal = total - occuranceTotal + .001
  for i in ranges:
    nonOccurance = counts[i]-occurances[i] + .0001
    pOfOccur = occurances[i]*100/occuranceTotal

    pOfNonOccur = nonOccurance*100/nonOccuranceTotal
      #print(pOfOccur, pOfNonOccur)

    if pOfOccur == 0:
      woe[i]=0
    else:
      woe[i]= math.log(pOfOccur/pOfNonOccur)

    iv[i]= (pOfOccur-pOfNonOccur)*woe[i]/100

  for i in range(total):
    for rng in ranges:
      if lst[i]<=rng:
        lst[i]=iv[rng]
        break
  return lst

#3D share

In [4]:
sa=.2
sb=.33
sc=.4

sxy = np.array(
    [[1.0,sa,0,0],
    [sb,sa*sb+1,0,0],
    [sc,sa*sc+1,1,0],      
    [0,0,0,1]]
)

syz = np.array(
    [[1.0,sa,sa+sa*sb,0],
    [0,1,sb,0],
    [0,sc,sa*sc+1,0],      
    [0,0,0,1]]
) 

szx = np.array(
    [[1+sa*sc,0,sa,0.0],
    [sb+sb*sc,1,sb,0],
    [sc,0,1,0],
    [0,0,0,1]]
)


In [5]:
def sharing(trip):
  #print(trip)
  tp_arr = trip.transpose()
  #print(tp_arr)
  one = np.ones(tp_arr[0].shape)
  #print(one)
  tp_arr = np.concatenate((tp_arr,[one]))
  #print(tp_arr)

  a = np.array(np.matrix(sxy)*np.matrix(tp_arr))
  a = np.array(np.matrix(syz)*np.matrix(a))
  a = np.array(np.matrix(szx)*np.matrix(a))
  
  #print(a)

  arr = np.array([a[0],a[1],a[2]])
  tp_arr = arr.transpose()
  #print(tp_arr)

  return tp_arr


In [6]:
def _3DShar(data):
  size = data.shape[1]
  reminder = size%3
  #print(size,reminder)

  #first triplate
  triplate = data[:,:3]
  #print(triplate)
  merged_array = np.array(sharing(triplate))
  for i in range(3,size-reminder,3):
    triplate = data[:,i:3+i]
    Rotated = np.array(sharing(triplate))
    # Merge the arrays along the columns
    merged_array = np.concatenate((merged_array, Rotated), axis=1)

  #Rotate if any colums are left
  if reminder!=0:
    triplate = data[:,-3:]
    Rotated = np.array(sharing(triplate))
    merged_array = np.concatenate((merged_array, Rotated[:,-1*(reminder):]), axis=1)
    #print(merged_array.shape)
  return merged_array

#Datasets

In [7]:
df = pd.read_csv('data.csv')

#change the colum with comannds.txt

In [8]:
#Nan_Removed csv file dependent
clas = df['Class']
df = df.drop(columns=['Class'])


In [9]:
lebels = df.columns
k = int(input("Enter K: "))
A = df.to_numpy()
svdDf = svdd(A,k)
df = pd.DataFrame(svdDf,columns=lebels)

for i in df:
  print(i)
  woe_iv = WOE_IV(list(df[i]),clas,1)
  df[i]=woe_iv


Unnamed: 1
Unnamed: 2
Unnamed: 3
Unnamed: 4
Unnamed: 5
Unnamed: 6
Unnamed: 7
Unnamed: 8
Unnamed: 9
Unnamed: 10
Unnamed: 11
Unnamed: 12
Unnamed: 13
Unnamed: 14
Unnamed: 15
Unnamed: 16
Unnamed: 17
Unnamed: 18
Unnamed: 19
Unnamed: 20
Unnamed: 21
Unnamed: 22
Unnamed: 23
Unnamed: 24
Unnamed: 25
Unnamed: 26
Unnamed: 27
Unnamed: 28
Unnamed: 29
Unnamed: 30
Unnamed: 31
Unnamed: 32
Unnamed: 33
Unnamed: 34
Unnamed: 35
Unnamed: 36
Unnamed: 37
Unnamed: 38
Unnamed: 39
Unnamed: 40
Unnamed: 41
Unnamed: 42
Unnamed: 43
Unnamed: 44


In [10]:
scaler = MinMaxScaler(feature_range=(0, 1))
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [11]:
A = df_normalized.to_numpy()
#print(A)
final_dataset = _3DShar(A)
print(final_dataset.shape)
df = pd.DataFrame(final_dataset)
df['Class'] = np.array(list(clas))
df.to_csv("6. SVD+WOE+IV+3dshare.csv",index=False)
#print(final_dataset)

(267, 44)
